In [10]:
# This is the generic code for health impact assessment (HIA) and burden of disease (BoD) calculations
# Forked originally from http://en.opasnet.org/w/HIA
# Original R code described in more detail at https://github.com/jtuomist/ghg-notebooks/wiki/Health-impact-assessment

import pandas as pd
import numpy as np

gas = ['CO2','CH4']

In [85]:
def concentration():
    df = pd.DataFrame({
        'exposure_agent':gas,
        'result':[1,2] # mg/kg, ppm, or similar
    })
    return df.set_index('exposure_agent')

def consumption():
    df = pd.DataFrame({
        'exposure_agent':gas,
        'result':[2,3]
    })
    return df.set_index('exposure_agent')

def exposure(
    p1 = concentration(),
    p2 = consumption()):
    
    out = p1 * p2
    #colnames(out@output)[colnames(out@output) == "Pathogen"] <- "Exposure_agent"

    return out

def bw():
    df = pd.DataFrame({
        'result':[70]
    })
    return df

def dose(
    p1 = exposure(),
    p2 = bw()):
    
    out = p1
    out['tmp'] = 1
    out = out.set_index('tmp', append=True)
    p2['tmp'] = 1
    p2['exposure_agent'] = 'CO2'
    p2 = p2.set_index(['exposure_agent','tmp'], append=True)
#    outbw = out * p2
#    outbw['scaling'] = 'bw'
#    outlog = np.log10(out).copy()
#    outlog['scaling'] = 'log10'
#    out['scaling'] = 'none'
#    out = out.append(outbw).append(outlog).reset_index()
    
    return out#.set_index(['exposure_agent','scaling'])

display(dose())
    
#dose
#function (...)
#{
#out <- exposure
#outbw <- out/BW
#outbw$Scaling <- "BW"
#outlog <- log10(out)
#outlog$Scaling <- "Log10"
#out$Scaling <- "None"
#out <- OpasnetUtils::combine(out, outbw, outlog)
#return(out)
#}

,,result
exposure_agent,tmp,
CO2,1,2
CH4,1,6
